In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor

In [2]:
%%time
df = pd.read_pickle("../data/interim/merged_raw_data.pkl")

CPU times: user 2.91 s, sys: 3.16 s, total: 6.07 s
Wall time: 6.37 s


In [3]:
df = df.set_index(["id", "d"])

In [4]:
TARGET = "sales"
FEATURES = [col for col in df.columns if col not in ["id", "date", "d", "data"] + [TARGET]]

In [54]:
TRAIN_START = 0
TRAIN_END = 365
VAL_START = 366
VAL_END = 365 + 28

In [62]:
train = df[(df.index.get_level_values(1) >= TRAIN_START) & (df.index.get_level_values(1) <= TRAIN_END)]
train.shape

(11128850, 21)

In [63]:
val = df[(df.index.get_level_values(1) >= VAL_START) & (df.index.get_level_values(1) <= VAL_END)]
val.shape

(853720, 21)

In [58]:
train["date"].nunique()

365

In [59]:
val["date"].nunique()

28

In [12]:
test = df[df["data"] == "val"]

In [64]:
X_tr = train[FEATURES]
X_val = val[FEATURES]
X_test = test[FEATURES]

In [65]:
y_tr = train[TARGET]
y_val = val[TARGET]
y_test = test[TARGET]

In [73]:
model = LGBMRegressor(n_estimators = 50,
                      learning_rate = 0.1,
                      num_leaves = 2**5 - 1,
                      max_bin = 100,
                      subsample = 0.5,
                      colsample = 0.5,
                      n_jobs = -1,
                      silent = -1,
                      verbose = -1)

In [74]:
model.fit(X_tr, y_tr, 
          eval_set = [(X_tr, y_tr), (X_val, y_val)], 
          eval_metric = 'l2',
          verbose = 10,
          early_stopping_rounds = 100)

Training until validation scores don't improve for 100 rounds
[10]	training's l2: 7.19025	valid_1's l2: 8.77016
[20]	training's l2: 5.89809	valid_1's l2: 7.50397
[30]	training's l2: 5.44618	valid_1's l2: 7.11556
[40]	training's l2: 5.2092	valid_1's l2: 6.90378
[50]	training's l2: 5.04308	valid_1's l2: 6.75607
Did not meet early stopping. Best iteration is:
[50]	training's l2: 5.04308	valid_1's l2: 6.75607


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample=0.5,
              colsample_bytree=1.0, importance_type='split', learning_rate=0.1,
              max_bin=100, max_depth=-1, min_child_samples=20,
              min_child_weight=0.001, min_split_gain=0.0, n_estimators=50,
              n_jobs=-1, num_leaves=31, objective=None, random_state=None,
              reg_alpha=0.0, reg_lambda=0.0, silent=-1, subsample=0.5,
              subsample_for_bin=200000, subsample_freq=0, verbose=-1)

Step 1
- Save X/y data
- CV generator
- run model
- predict test set
- restructure submission file

Step 2
- correct metric(wrmsse)
- hyperparameter optimization
- feature engineering